In [ ]:
import pandas as pd
from django.core.exceptions import ObjectDoesNotExist
from archiv.utils import create_bestand

In [ ]:
# file = "legacy_data/WStLA_MD_Allgemeine_Registratur_Akten_gesamt_20180903.xlsx"
file = "legacy_data/20180901_VGA.xlsx"
res_type_scheme, _ = SkosConceptScheme.objects.get_or_create(dc_title='res_type')

In [ ]:
df = pd.read_excel(file, converters={'Datum original': str}).fillna('False')

In [ ]:
df

In [ ]:
for i, row in df.iterrows():
    item = None
    print(i)
    signatur = "|".join(
        [
            str(row.get('(Archiv)Signatur', default = "Signatur")), 
            str(row.get('Karton', default='Karton')), 
            str(row.get('Aktenzahl', default='Aktenzahl'))
        ]
    )
    if row['Teil von Bestand'] != 'False':
        bestand = create_bestand(row['Teil von Bestand'])[-1]
        item, _ = ArchResource.objects.get_or_create(
            signature=signatur
        )
        item.location = bestand
        item.save()
        if row['Betreff (Titel)'] != 'False':
            item.title = row['Betreff (Titel)']
        if row['Zusammenfassung'] != 'False':
            item.abstract = row['Zusammenfassung']
        if row['Kommentar'] != 'False':
            item.notes = row['Kommentar']
        if row['Datum original '] != 'False':
            item.written_date = row['Datum original '].replace('00:00:00', '')
        if row['Datum normalisiert DD/MM/YYYY)'] != 'False':
            try:
                item.not_before = pd.to_datetime(row['Datum normalisiert DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Datum normalisiert nicht nach(DD/MM/YYYY)'] != 'False':
            try:
                item.not_after = pd.to_datetime(row['Datum normalisiert nicht nach (DD/MM/YYYY)'])
            except Exception as e:
                print(e)
        if row['Typ'] != 'False':
            type_vocab, _ = SkosConcept.objects.get_or_create(
                pref_label="{}".format(row['Typ'])
            )
            type_vocab.scheme.add(res_type_scheme)
            item.res_type = type_vocab
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Orte'] != 'False':
            for x in row['Orte'].split(';'):
                try:
                    pl = Place.objects.get(name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_place.add(pl)
        if row['Institutionen'] != 'False':
            for x in row['Institutionen'].split(';'):
                try:
                    pl = Institution.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_inst.add(pl)
        if row['Personen'] != 'False':
            for x in row['Personen'].split(';'):
                try:
                    pl = Person.objects.get(written_name=x.strip())
                except Exception as e:
                    pl = None
                    print(e, x.strip())
                if pl:
                    item.mentioned_person.add(pl)
    if item:
        item.save()

In [ ]:
# RepoLocation.objects.all().delete()